In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from vastai.api import VastClient
from io import StringIO
import time
from IPython.display import clear_output

In [3]:
vast = VastClient().authenticate()

api_key_file:  ~/.vast_api_key
Initializing vast.ai client with api_key from /home/sam/.vast_api_key.
Already logged in.


In [4]:
def monitor_file(log_file, popen=None, n_lines=5, update_freq_s=1):
    while True:
        try:
            clear_output(wait=True)
            print(remote['tail'](log_file,'-n%i'%n_lines))
            if popen is not None:
                ret = popen.proc.poll()
                if ret is not None:
                    print("Process has exited with code:",popen.proc.poll())
                    break
            time.sleep(update_freq_s)
        except KeyboardInterrupt:
            break
        except NameError:
            break

## Search for Available Machines

In [15]:
def filter_offers():
    return [offer for offer in vast.search_offers('dph', 'dph<.12 inet_down>200', instance_type='on-demand') 
        if offer['gpu_name'] and offer['id'] not in []]

In [ ]:
for offer in filter_offers():
    print(offer['id'], offer['dph_total'], offer['gpu_name'], offer['inet_down'], offer['inet_down'])

## Start new Instance with Mongodb

In [40]:
offer = 352306
# offer = filter_offers()[0]['id']
onstart = """#!/bin/sh
touch ~/.no_auto_tmux
echo onstart.sh Starting `date` >> /root/startup.log 2>&1
apt-get install -y mongodb-server git vim >> /root/startup.log 2>&1
service mongodb start >> /root/startup.log 2>&1
pip install --upgrade pip pipenv >> /root/startup.log 2>&1
echo onstart.sh Completed at `date` >> /root/startup.log
"""
resp=vast.create_instance(offer, disk=20, onstart_cmd=onstart, python_utf8=True, lang_utf8=True,
                          image="tensorflow/tensorflow:1.15.0-gpu-py3-jupyter")
instance_id = resp['new_contract']

instance = vast.get_instance(instance_id, retries=4).wait_until_running()
remote=instance.pb_remote

https://vast.ai/api/v0/asks/352306/?api_key=e794a3273e5a3e74d545bbc368849ff263f726042cf92587df9519a6cda62e74 
 {"client_id": "me", "image": "tensorflow/tensorflow:1.15.0-gpu-py3-jupyter", "price": null, "disk": 20, "label": null, "onstart": "#!/bin/sh\ntouch ~/.no_auto_tmux\necho onstart.sh Starting `date` >> /root/startup.log 2>&1\napt-get install -y mongodb-server git vim >> /root/startup.log 2>&1\nservice mongodb start >> /root/startup.log 2>&1\npip install --upgrade pip pipenv >> /root/startup.log 2>&1\necho onstart.sh Completed at `date` >> /root/startup.log\n", "runtype": "ssh", "python_utf8": true, "lang_utf8": true, "use_jupyter_lab": false, "jupyter_dir": null, "create_from": null, "force": false}
{"success": true, "new_contract": 403437}
Waiting 60s...
Waiting 60s...
Waiting 60s...


In [41]:
print(remote['tail']('/var/log/mongodb/mongodb.log','-n2'))

2019-10-31T06:48:40.680+0000 I FTDC     [initandlisten] Initializing full-time diagnostic data capture with directory '/var/lib/mongodb/diagnostic.data'
2019-10-31T06:48:40.680+0000 I NETWORK  [initandlisten] waiting for connections on port 27017



In [42]:
print(instance.ssh_connection_command)

ssh root@ssh4.vast.ai -p 13437 -i /home/sam/.ssh/vastai


In [43]:
# onstart = "#!/bin/sh\n"\
#           "touch ~/.no_auto_tmux\n"\
#           "apt-get install -y mongodb-server\n"\
#           "mkdir -p ~/.ssh\n"\
#           "ssh-keygen  -P '' -f ~/.ssh/mongo\n"
# resp=vast.create_instance(filter_offers()[0]['id'], disk=1, onstart_cmd=onstart,
#                           image="tensorflow/tensorflow:1.14.0-gpu-py3-jupyter")
# worker_instance_id = resp['new_contract']

# time.sleep(30)
# worker_instance = vast.get_instance(worker_instance_id)

## Install model and download data

In [44]:
# instance=vast.get_instance(403430)
# remote=instance.pb_remote
# print(instance.ssh_connection_command)

In [45]:
install_script_target='/root/install_script.sh'
install_script = StringIO("""#!/bin/sh
INSTALL_LOG=/root/install.log
echo avian_vocalizations INSTALL BEGIN `date` >> $INSTALL_LOG 2>&1
cd ~
[ ! -d avian_vocalizations ] && git clone https://github.com/samhiatt/avian_vocalizations.git  >> $INSTALL_LOG 2>&1
cd avian_vocalizations
export LC_ALL=C.UTF-8
export LANG=C.UTF-8
pipenv lock -r > requirements.txt
pip install -r requirements.txt >> $INSTALL_LOG 2>&1
download_data >> $INSTALL_LOG 2>&1
echo avian_vocalizations INSTALL COMPLETE `date` >> INSTALL_LOG 2>&1
""")
# ("""pipenv install >> $INSTALL_LOG 2>&1
# pipenv run python setup.py develop >> $INSTALL_LOG 2>&1
# pipenv run download_data >> $INSTALL_LOG 2>&1
# echo avian_vocalizations INSTALL COMPLETE `date` >> INSTALL_LOG 2>&1
# """)
# monitor_output(mongo_instance, install_script)
remote.sftp.putfo(install_script,install_script_target)
remote['chmod']('+x',install_script_target)
p=remote.session().popen(install_script_target)
p

In [46]:
monitor_file('/root/install.log', p)

werkzeug==0.16.0
wget==3.2
wheel==0.33.6 ; python_version >= '3'
wrapt==1.11.2
/root/install_script.sh: 11: /root/install_script.sh: download_data: not found

Process has exited with code: 0


In [57]:
remote = instance.pb_remote

In [63]:
hyperopt_worker_log="/root/hyperopt_worker.log"
start_hyperopt_worker_target='/root/avian_vocalizations/start_hyperopt_worker.sh'
start_hyperopt_worker="#!/bin/sh\ncd /root/avian_vocalizations\n"\
    "export LC_ALL=C.UTF-8\n"\
    "export LANG=C.UTF-8\n"\
    "nohup hyperopt-mongo-worker --mongo=localhost:27017/avian_vocalizations "\
    " --max-consecutive-failures=1 --reserve-timeout=999999 --workdir=."\
    " >> %s 2>&1\n"%hyperopt_worker_log

print(start_hyperopt_worker)
remote.sftp.putfo(StringIO(start_hyperopt_worker),start_hyperopt_worker_target)
remote['chmod']('+x',start_hyperopt_worker_target)
pworker=remote.session().popen(start_hyperopt_worker_target)

#!/bin/sh
cd /root/avian_vocalizations
export LC_ALL=C.UTF-8
export LANG=C.UTF-8
nohup hyperopt-mongo-worker --mongo=localhost:27017/avian_vocalizations  --max-consecutive-failures=1 --reserve-timeout=999999 --workdir=. >> /root/hyperopt_worker.log 2>&1



In [64]:
monitor_file(hyperopt_worker_log, pworker)

INFO:hyperopt.mongoexp:no job found, sleeping for 1.4s
INFO:hyperopt.mongoexp:no job found, sleeping for 2.5s
INFO:hyperopt.mongoexp:no job found, sleeping for 3.0s
INFO:hyperopt.mongoexp:no job found, sleeping for 2.4s
INFO:hyperopt.mongoexp:no job found, sleeping for 1.9s



In [ ]:
# vast.stop_all_instances()

In [62]:
list(remote.pgrep('hyperopt'))

[]